# Exercise 3: Dynamic Page Web crawling
Retrieve the average prices of intercity bus connections between the largest k = 5, 10, 50
German cities based on the website
https://global.flixbus.com/.

*Add on:* Visualize this data as a network plot.

### How to get the largest Cities:
Destatis has that data in xlsx format for all german cities:
https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/05-staedte.html

In [24]:
#Load the top n cities to do this:
import pandas as pd
df = pd.read_excel("05-staedte.xlsx",1,header=5).dropna(how="any")

def get_city_name(row:pd.Series)->pd.Series:
    row[14] = row[7].split(",")[0]
    return row
df = df.apply(get_city_name,axis=1)
df = df.rename(columns={1:"nr",2:"land",3:"rb",4:"kreis",5:"verb",6:"gem",7:"stadtbezeichnung",8:"plz",9:"flaeche",10:"einwohner",11:"mann",12:"frau",13:"dichte",14:"stadt"})
n = 5
top_n_df = df.sort_values(by="einwohner",ascending=False).head(n)
top_n_df


,nr,land,rb,kreis,verb,gem,stadtbezeichnung,plz,flaeche,einwohner,mann,frau,dichte,stadt
1,1,11.0,0.0,0.0,0.0,0.0,"Berlin, Stadt",10178.0,891.12,3664088.0,1802038.0,1862050.0,4112.0,Berlin
2,2,2.0,0.0,0.0,0.0,0.0,"Hamburg, Freie und Hansestadt",20038.0,755.09,1852478.0,906933.0,945545.0,2453.0,Hamburg
3,3,9.0,1.0,62.0,0.0,0.0,"München, Landeshauptstadt",80313.0,310.70,1488202.0,725905.0,762297.0,4790.0,München
4,4,5.0,3.0,15.0,0.0,0.0,"Köln, Stadt",50667.0,405.01,1083498.0,528075.0,555423.0,2675.0,Köln
5,5,6.0,4.0,12.0,0.0,0.0,"Frankfurt am Main, Stadt",60311.0,248.31,764104.0,377199.0,386905.0,3077.0,Frankfurt am Main


### How to define the average price between these cities? 
I define the average price as the average price a customer has to pay if he wants to travel within the next two weeks with flixbus as the price seems to differ based on the weekday and on how much in advance a customer books a ticket. Two weeks seems to be enough of a time to cancel out the premium price for late bookings and also seems to cancel out weekday-dependent price differences

Direction needs to be taken into account as the price differs between directions

In [65]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Firefox()
driver.get("https://www.flixbus.de/")


In [66]:
#Get rid of cookie modal. selenium had its isssues. this is the only workaround i could find...
webdriver.ActionChains(driver).move_by_offset(475,500).click().perform()

In [59]:
btns= driver.find_elements(by=By.XPATH,value="//button")

for btn in btns:
    print(btn.text)
#btns[2].click()
#/div/div/div/div/div[2]/div/div[2]/div/div/div/button[1]
driver.find_elements(By.XPATH,value = "//iframe")[0].click()



Verbindung suchen


ElementNotInteractableException: Message: Element <iframe id="uc-cross-domain-bridge" src="https://app.usercentrics.eu/browser-sdk/4.0.0/cross-domain-bridge.html"> could not be scrolled into view
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
ElementNotInteractableError@chrome://remote/content/shared/webdriver/Errors.jsm:293:5
webdriverClickElement@chrome://remote/content/marionette/interaction.js:156:11
interaction.clickElement@chrome://remote/content/marionette/interaction.js:125:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:203:24
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:91:31
